# 1. Load Corpse

In [3]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  9924k      0 --:--:-- --:--:-- --:--:-- 10.1M


In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
len(text)

1115394

In [8]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



# 2. Prepare Datasets

In [9]:
# Vocabulary
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(''.join(vocab))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
# Encode and Decode function
stoi = { ch: i for i, ch in enumerate(vocab) }
itos = { i: ch for i, ch in enumerate(vocab) }
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("hii python"))
print(decode(encode("hii python")))


[46, 47, 47, 1, 54, 63, 58, 46, 53, 52]
hii python


In [12]:
# Training and validation data
import torch

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [15]:
# Batches
import torch

torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split: str):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    xb = torch.stack([data[i:i+block_size] for i in ix])
    yb = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return xb, yb

xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When the inputs are: {context.tolist()}, target: {target}")

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When the inputs are: [24], target: 43
When the inputs are: [24, 43], target: 58
When the inputs are: [24, 43, 58], target: 5
When the inputs are: [24, 43, 58, 5], target: 57
When the inputs are: [24, 43, 58, 5, 57], target: 1
When the inputs are: [24, 43, 58, 5, 57, 1], target: 46
When the inputs are: [24, 43, 58, 5, 57, 1, 46], target: 43
When the inputs are: [24, 43, 58, 5, 57, 1, 46, 43], target: 39
When the inputs are: [44], target: 53
When the inputs are: [44, 53], target: 56
When the inputs are: [44, 53, 56], target: 1
When the inputs are: [44, 53, 56, 1], target: 58
When the inputs are: [44

# 3. BiGram

In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BiGramLanguageModel(nn.Module):

    def __init__(self, vocab_size) -> None:
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None): # idx is (B, T)
        logits = self.token_embedding_table(idx) # logits is (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # To fulfill PyTorch cross entropy requirements
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # PyTorch requires (B, C, T)
        return logits, loss

    def generate(self, idx, num_examples):
        for _ in range(num_examples):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, C). Only needs the last result of block_size (T)
            probs = F.softmax(logits, dim=-1) # (B, C). Softmax on dimension C
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

        

m = BiGramLanguageModel(vocab_size=vocab_size)
xb, yb = get_batch('train')
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# We see the loss on the randomly inited bigram LM is close to uniformly distributed loss
print(-torch.log(torch.tensor(1 / vocab_size)))

# Inference
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), num_examples=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7051, grad_fn=<NllLossBackward0>)
tensor(4.1744)

pxMHoRFJa!JKmRjtXzfN:CERiC-KuDHoiMIB!o3QHN
,SPyiFhRKuxZOMsB-ZJhsucL:wfzLSPyZalylgQUEU cLq,SqV&vW:hhi


In [40]:
# Training
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

def train(epoch: int):
    for _ in range(epoch):
        xb, yb = get_batch('train')
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True)

        loss.backward()
        optimizer.step()

    return loss.item()

print(train(1000))

2.658010721206665


In [42]:
# We see the pattern looks similar to Shakespeare
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), num_examples=500)[0].tolist()))


AU,
Gowist Weano th t?EX&jFumat the of laticond ionedrt, cIce ce n thive'dZKIve, avind n, shadYZTH!abIJ&--cothere, m;
Tr myoman iss no t Fk cois The fy.Ebe hysoimay atode isevV:ZUAElanon ishuromV&Qxl?ughen,

TRT:
Pl, nopomngoreppurkRCKpo, me s ft he te s I'tun tav&
Bnd or o ft ges, IN.

WARSio. IVT:
SSivk!
Hand, fatrarcugeQG, he ame s mymenosos and DKpue whea abvpangmymy dvin
CJins wW:CJm bYWCkSise mye adettatlyxMOLWAscotheirilknes thethin w,S$

IUzord;Wthimy my by ithes etavery y: hal; t d.
NYw
